Perfect — here’s your full setup for **`email_script_cogsci.py`**, which:

---

### ✅ Features

* Searches **PubMed**, **Semantic Scholar**, and **RSS feeds**
* Filters for cognitive science keywords like `"cognitive"`, `"reasoning"`, `"neural network"`, etc.
* Outputs results in a **`pandas` DataFrame** when `TEST_MODE = True`
* Sends a **monthly digest email** when `TEST_MODE = False`
* Tracks seen items in a **separate file**: `seen_articles_cogsci.json`

---

### 🧠 Cognitive Science Sources Used

* **PubMed**: psychology, neuroscience, computational modeling
* **Semantic Scholar**: open access only
* **RSS Feeds**:

  * *Cognitive Science* (Wiley): [https://onlinelibrary.wiley.com/rss/journal/15516709](https://onlinelibrary.wiley.com/rss/journal/15516709)
  * *Trends in Cognitive Sciences* (Elsevier): [https://rss.sciencedirect.com/publication/science/13646613](https://rss.sciencedirect.com/publication/science/13646613)
  * *Frontiers in Psychology – Cognitive Science*: [https://www.frontiersin.org/journals/psychology/sections/cognitive-science/rss](https://www.frontiersin.org/journals/psychology/sections/cognitive-science/rss)

---

### ✅ Full Script: `email_script_cogsci.py`

```python

```


In [ ]:
import os
import json
import smtplib
import requests
import feedparser
import pandas as pd
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from Bio import Entrez

# === Config ===
Entrez.email = "your_email@example.com"

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_ADDRESS = "your_email@gmail.com"
EMAIL_PASSWORD = "your_app_password"
TO_EMAIL = "recipient@example.com"

MAX_RESULTS = 10
SEEN_FILE = "seen_articles_cogsci.json"
TEST_MODE = True  # Set to False to send email

# === Keywords to filter RSS content ===
KEYWORDS = ["cognitive", "reasoning", "language", "neural", "attention", "memory", "learning", "brain", "decision making", "consciousness"]

# === RSS Feeds for Cognitive Science ===
RSS_FEEDS = {
    "Cognitive Science (Wiley)": "https://onlinelibrary.wiley.com/rss/journal/15516709",
    "Trends in Cognitive Sciences": "https://rss.sciencedirect.com/publication/science/13646613",
    "Frontiers in Psychology – Cognitive Science": "https://www.frontiersin.org/journals/psychology/sections/cognitive-science/rss"
}

def load_seen():
    if os.path.exists(SEEN_FILE):
        with open(SEEN_FILE, "r") as f:
            return set(json.load(f))
    return set()

def save_seen(seen_set):
    with open(SEEN_FILE, "w") as f:
        json.dump(list(seen_set), f)

def search_pubmed(term, max_results=10):
    handle = Entrez.esearch(
        db="pubmed",
        term=term,
        retmax=max_results,
        sort="relevance",
        datetype="pdat",
        reldate=30
    )
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

def fetch_pubmed_details(id_list):
    ids = ",".join(id_list)
    handle = Entrez.efetch(db="pubmed", id=ids, retmode="xml")
    records = Entrez.read(handle)
    handle.close()
    results = []
    for article in records['PubmedArticle']:
        pmid = article['MedlineCitation']['PMID']
        title = article['MedlineCitation']['Article']['ArticleTitle']
        link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
        authors = article['MedlineCitation']['Article'].get('AuthorList', [])
        author_str = ', '.join(
            [f"{a.get('ForeName', '')} {a.get('LastName', '')}".strip() for a in authors if 'LastName' in a]
        )
        results.append({
            "Title": title,
            "Authors": author_str,
            "Source": "PubMed",
            "PDF": link,
            "ID": f"pmid:{pmid}"
        })
    return results

def fetch_semantic_scholar(query="cognitive science"):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": 15,
        "fields": "title,authors,url,isOpenAccess,openAccessPdf"
    }
    response = requests.get(url, params=params)
    data = response.json()
    results = []
    for paper in data.get("data", []):
        if not paper.get("isOpenAccess"):
            continue
        title = paper.get("title", "No Title")
        url = paper.get("openAccessPdf", {}).get("url") or paper.get("url")
        authors = ', '.join([a['name'] for a in paper.get("authors", [])])
        paper_id = paper.get("paperId", "")
        results.append({
            "Title": title,
            "Authors": authors,
            "Source": "Semantic Scholar",
            "PDF": url,
            "ID": f"sem:{paper_id}"
        })
    print(f"✅ Semantic Scholar returned {len(results)} open-access results.")
    return results

def fetch_rss_articles():
    articles = []
    for source, url in RSS_FEEDS.items():
        print(f"🔎 Fetching from {source}...")
        feed = feedparser.parse(url)
        print(f"➡️ {len(feed.entries)} entries fetched.")
        for entry in feed.entries:
            title = entry.get('title', '')
            link = entry.get('link', '')
            authors = entry.get('author', entry.get('dc_creator', 'Unknown Author'))
            if any(keyword in title.lower() for keyword in KEYWORDS):
                articles.append({
                    "Title": title,
                    "Authors": authors,
                    "Source": source,
                    "PDF": link,
                    "ID": f"rss:{link}"
                })
    print(f"✅ RSS filtering yielded {len(articles)} total matches.")
    return articles

def format_html(articles):
    html = "<html><body><h2>Monthly Cognitive Science Research Digest</h2><ol>"
    for a in articles:
        html += f"<li><b>{a['Title']}</b><br><small>{a['Authors']}</small><br>"
        html += f"<a href='{a['PDF']}'>{a['PDF']}</a> <i>({a['Source']})</i><br><br></li>"
    html += f"</ol><p style='font-size:small;'>Generated {datetime.now().strftime('%Y-%m-%d %H:%M')}</p></body></html>"
    return html

def send_email(subject, html_body):
    msg = MIMEMultipart("alternative")
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = TO_EMAIL
    msg['Subject'] = subject
    msg.attach(MIMEText(html_body, "html"))
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

if __name__ == "__main__":
    try:
        seen = load_seen()

        pubmed_ids = search_pubmed(
            '("cognitive science" OR "cognitive modeling" OR "neural computation" OR "computational neuroscience") AND free full text[Filter]',
            max_results=15
        )
        pubmed_results = fetch_pubmed_details(pubmed_ids)
        sem_results = fetch_semantic_scholar()
        rss_results = fetch_rss_articles()

        all_results = pubmed_results + sem_results + rss_results
        new_articles = [a for a in all_results if a["ID"] not in seen]
        digest = new_articles[:MAX_RESULTS]

        if TEST_MODE:
            if digest:
                df = pd.DataFrame(digest)
                pd.set_option('display.max_colwidth', 150)
                print("\n🧠 New Cognitive Science Articles:\n")
                print(df[["Title", "Authors", "Source", "PDF"]])
            else:
                print("✅ Script ran successfully, but no new matching articles found.")
        else:
            if digest:
                html = format_html(digest)
                send_email("Monthly Cognitive Science Research Digest", html)
                seen.update([a["ID"] for a in digest])
                save_seen(seen)

    except Exception as e:
        print(f"❌ Error occurred: {e}")


---

### ✅ Getting Started

1. Save this as `email_script_cogsci.py`
2. Run locally to test:

   ```bash
   python email_script_cogsci.py
   ```
3. Flip `TEST_MODE = False` when ready for email delivery
4. Use Task Scheduler (Windows) or cron (Linux/macOS) for monthly runs

---

Would you like:

* Abstracts shown in the digest?
* A version that also saves a `.csv` or `.html` file locally?

You're now set up to monitor cross-disciplinary advances in cognitive science, psychology, neuroscience, and AI from top sources!
